### This is an notebook to research all properties of syntax trees, including:
Finding association rules

Filling in the blank on missing word classes

Building trees

In [1]:
import re
import nltk
import math
import requests
from bs4 import BeautifulSoup
from PyDictionary import PyDictionary

_d=PyDictionary()

In [189]:
x = 'The big man from n loving bagels with n'
#'The cat killed the dirty dog in the house because he threw nasty bread by the fire'
#"in the smelly and wholesome and brilliant and bubbly couch"
#"the brilliant and bubbly and whimsical ball"
p = nltk.parse.api.ParserI
_d=PyDictionary()

#Array forces ordering
syntaxRulesReverse = [
    ['(adjconj)+adj', 'adj'], 
    ['(advP)*adj', 'ajP'], 
    ['(advP)?adv', 'avP'], 
    ['PnP', 'pP'],
    ['(D)?(ajP)*N(pP)*(CP)?', 'nP'], 
    ['V(nP)+(pP)*', 'vP'], 
    ['(Nconj)+N', 'N'],
    ['(nPconj)+nP', 'nP'],
    ['nPvP', 'tP'],
    ['(tPconj)+tP', 'tP']]

In [17]:
_x = x.split()
_xd = ""
for token in _x:
    _xd += getClass(token)
_xd

/home/mkeays/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/mkeays/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word
Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Word given is not a valid English Word
Error: The Following Error occured: list index out of range
Error: The Word given is not a valid English Word


'DadjNPNVNPN'

In [31]:
tree = [_xd]
diagram = []
while needsParsing(tree[-1]):
    level = ""
    dlvl = []
    for rule in syntaxRulesReverse:
        m = re.search(rule[0], tree[-1])
        if m:
            level = tree[-1][0:m.start()] + rule[1] + tree[-1][m.end():]
            tree += [level]
            dlvl += [rule[1], tree[-1][m.start():m.end()], m.start(), m.end()]
            diagram += [dlvl]
            break
    if level == "":
        break
    print('.', end='')
[print(_, end='\n|\n') for _ in tree];None

........DadjNPNVNPN
|
DajPNPNVNPN
|
nPPNVNPN
|
nPPnPVNPN
|
nPpPVNPN
|
nPpPVnPPN
|
nPpPVnPPnP
|
nPpPVnPpP
|
nPpPvP
|


In [30]:
level

''

In [237]:
_search = re.search(tree[-2], syntaxRulesReverse[-1][0])
tree[-2][0:_search.start()] + syntaxRulesReverse[-1][1] + tree[-2][_search.end():]

'PP'

In [183]:
def needsParsing(x):
    if x != 'nP' and x != 'tP' and x != 'vP' and x != 'CP' and x!= 'pP':
        return True
    return False
def getClass(x):
    wC = 'X'
    #Meaning -- regular
    word = _d.meaning(x)
    if word is not None:
        wC = list(word.keys())[0]
    
    #Meaning -- google
    word = _d.googlemeaning(x)
    if word is not None:
        wC =  word.split(':')[1].split()[0]

    if wC.lower() == 'noun' or wC.lower() == 'pronoun':
        return 'N'
    elif wC.lower() == 'verb':
        return 'V'
    elif wC.lower() == 'adjective':
        return 'adj'
    elif wC.lower() == 'adverb':
        return 'adv'
    elif wC.lower() == 'determiner':
        return 'D'
    elif wC.lower() == 'preposition':
        return 'P'
    elif wC.lower() == 'conjunction':
        return 'conj'
        
    return wC

In [186]:
getClass('you')

/home/mkeays/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/mkeays/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


IndexError: list index out of range

# X-Bar and Array Tree Data Structures

In [2]:
### Getting parts of speach using wordsmyth
word = "cover"
url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
s = requests.Session()
res = s.get(url, verify = False)
soup = BeautifulSoup(res.text, 'html5lib')
soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text

/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'transitive verb'

In [9]:
#temporary static dict
localBank = {'is': 'v',
             'need' : 'v',
             'needs' : 'v',
             'do': 'v',
             'does': 'v',
             'I' : 'pn',
             'a' : 'd',
             'the' : 'd',
            #homework specific - no working suplementary dict
            'well': 'adj',
            'plays': 'v'}

In [10]:
def getLikelyClass(word, convert = False):
    print(word)
    if word in localBank:
        return localBank[word]
    url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
    s = requests.Session()
    res = s.get(url, verify = False)
    soup = BeautifulSoup(res.text, 'html5lib')
    txt = soup.findAll("table", {"class": "maintable"})[0].find_all('td', {"class": "data"})[0].a.text
    switch = {
        'determiner'          :        'd',
        'noun'                :        'n',
        'pronoun'             :        'pn',
        'preposition'         :        'p',
        'transitive verb'     :        'v', #vt
        'intransitive verb'  :         'v', #vi
        'ditransitive verb'  :         'v', #vd
        'adjective'           :        'adj',
        'adverb'              :        'adv',
        'conjunction'         :        'conj',
    }
    if convert and txt in switch:
        localBank[word] = switch[txt]
        return switch[txt]
    return txt

def defineLikelyClass(word, wordClass, force=False):
    if word not in localBank or force:
        localBank[word] = wordClass
        return True
    return False

In [11]:
getLikelyClass('\'s', True)

's


/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


IndexError: list index out of range

In [12]:
#defineLikelyClass('her', 'd', True)
x = 'The coach will be using a new approach'.split()
y = 'd n t v v d adj n'.split()
for i in range(len(x)):
    defineLikelyClass(x[i], y[i], True)

In [13]:
sentence = "The coach will be using a new approach"

#This will build all of the possible base cases for perspective
#based on research forensics for each given word
def constructPerspective(sentence):
    perspective = []
    tokens = [[_, getLikelyClass(_.lower(), True)] for _ in sentence.split(" ")]
    for token in tokens:
        perspective.append([token[1], [token[0]]])
    return perspective
x = constructPerspective(sentence)
print(x)

the
coach
will
be
using
a
new
approach
[['d', ['The']], ['n', ['coach']], ['t', ['will']], ['v', ['be']], ['v', ['using']], ['d', ['a']], ['adj', ['new']], ['n', ['approach']]]


In [40]:
perspectives = [x]
#I am Michael, I love Raquel, I eat protien, etc...
new_perspectives = []
exhaustive_perspectives = []
rules = {
    "vP" : [["vbar"], ["vP", "conj", "vP"]],
    "vbar" : [["vbar", "pP"], ["vbar", "advP"], ["advP", "vbar"], ["vbar", "dP"], ["v"], ["v", "dP"], ["v", "adjP"], ["v", "vP"]],
    "adjP" : [["adjbar"]],
    "adjbar" : [["adj"], ["advP", "adjbar"]],
    "advP" : [["advbar"]],
    "advbar" : [["adv"], ["advP", "advbar"]],
    "nbar": [["adjP", "nbar"], ["nbar", "pP"], ["n"], ["n", "pP"], ["pn"]],
    "nP" : [["nbar"], ["nP", "conj", "nP"]],
    "dbar" : [["d", "nP"], ["nP"]],
    "dP" : [["dP", "dbar"], ["dbar"], ["dP", "conj", "dP"]],
    "pbar" : [["p", "dP"]],
    "pP" : [["pbar"]],
    "negbar" : [["neg", "vP"]],
    "negP" : [["negbar"]],
    "t" : [["v"]],
    "tbar" : [["vP"], ["t", "vP"], ["t", "negP"]],
    "tP" : [["dP", "tbar"], ["tP", "conj", "tP"]],
    "cbar" : [["c", "tP"], ["tP"]],
    "cP" : [["cbar"], ["cP", "conj", "cP"]]
}

#Keep track of where matches for each rule occurs, when creating a new node,
#select the simplest matches first (adjbar before tbar), and the match that occurs later in the tree
mutating = True
while mutating:
    new_perspectives = []
    for inp in perspectives:
        matches = {}
        for rule, conditionSet in rules.items():    
                for conditions in conditionSet:
                    conditionMatch, conditionPosition = 0, 0
                    #print(rule + " : " + str(conditions))
                    for i in range(len(inp)):
                        node = inp[i]
                        if node[0] == conditions[conditionMatch]:
                            #print(rule + ", " + node[0])
                            if conditionMatch == 0:
                                conditionPosition = i
                            conditionMatch += 1
                            if conditionMatch == len(conditions):
                                if rule in matches:
                                    matches[rule] += [(conditionPosition, conditionPosition + conditionMatch)]
                                else:
                                    matches.update({rule:[(conditionPosition, conditionPosition + conditionMatch)]})
                                conditionMatch = 0
                        else:
                            conditionMatch = 0
        #print("{} -> {}".format(inp, matches))
        for matchLabel, matchSets in matches.items():
            for matchSet in matchSets:
                #print("{} :- {}".format(matchLabel, matchSet))
                subset = inp[matchSet[0]:matchSet[1]]
                new_perspective = inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:]
                if new_perspective not in new_perspectives:
                    new_perspectives.append(new_perspective)
        for perspective in perspectives:
            if perspective not in exhaustive_perspectives:
                exhaustive_perspectives.append(perspective)
            if inTreeTop(perspective, "cP", strict = True) == (True, True) and perspective not in new_perspectives:
                new_perspectives.append(perspective)
    if perspectives == new_perspectives or len(new_perspectives) == 0:
        mutating = False
        break
    perspectives = new_perspectives.copy()

In [41]:
#determines if the given rule is at the top of a given tree
#Strict disallows the tree from having different neighbors at the top
def inTreeTop(tree, rule, strict = False):
    retVal = False
    strictVal = True
    for topNode in tree:
        if topNode[0] == rule:
            retVal = True
        else:
            strictVal = False
    if strict:
        return (retVal, strictVal)
    return retVal
def pruneTrees(trees):
    pruned = []
    for tree in trees:
        finishedVal = inTreeTop(tree, "cP", strict = True)
        if finishedVal[0] and finishedVal[1]:
            pruned.append(tree)
    return pruned
final_perspectives = pruneTrees(perspectives)

In [38]:
print("Trees: {}".format(len(final_perspectives)))
print(treeHeight(final_perspectives[0]))
final_perspectives[1]

Trees: 2
13


[['cP',
  [['cbar',
    [['tP',
      [['dP',
        [['dbar', [['d', ['The']], ['nP', [['nbar', [['n', ['coach']]]]]]]]]],
       ['tbar',
        [['t', ['will']],
         ['vP',
          [['vbar',
            [['v', ['be']],
             ['vP',
              [['vbar',
                [['vbar', [['v', ['using']]]],
                 ['dP',
                  [['dbar',
                    [['d', ['a']],
                     ['nP',
                      [['nbar',
                        [['adjP', [['adjbar', [['adj', ['new']]]]]],
                         ['nbar',
                          [['n', ['approach']]]]]]]]]]]]]]]]]]]]]]]]]]]]]

In [341]:
latexKeys = {
    "xbar" : '$\\bar{{{}}}$'
}

def printTree(tree):
    if type(tree) == str:
        return tree + " "
    t = tree[0]
    if type(t) == str:
        return t + " "
    label = t[0]
    if len(label) == 1:
        label = label.upper()
    ploc = label.find('P')
    if ploc > 0:
        if len(label[:ploc]) == 1:
            label = label.upper()
    barloc = label.find('bar')
    if barloc > 0:
        sublabel = label[:barloc]
        if len(sublabel) == 1:
            sublabel = sublabel.upper()
        label = latexKeys["xbar"].format(sublabel)
    if len(t[1]) == 1:
        return '[.{} {}]'.format(label, printTree(t[1]))
    return '[.{} \n\t{} \n\t{}]'.format(label, printTree([t[1][0]]), printTree([t[1][1]]))
    
print(printTree(final_perspectives[1]))

[.CP [.$\bar{C}$ [.TP 
	[.DP [.$\bar{D}$ [.NP [.$\bar{N}$ [.pn She ]]]]] 
	[.$\bar{T}$ 
	[.T might ] 
	[.VP [.$\bar{V}$ 
	[.$\bar{V}$ [.V play ]] 
	[.DP [.$\bar{D}$ [.NP [.$\bar{N}$ [.N tomorrow ]]]]]]]]]]]


In [25]:
#Gives the height of a tree, for those from perspective
def treeHeight(tree):
    if type(tree) == str:
        return 0
    if len(tree) == 1:
        if type(tree[0]) == str:
            return 0
    return 1+max([treeHeight(_[1]) for _ in tree])
    #return max([(0 if len(_) == 1 else max(treeHeight(__) for __ in _[1]) if (type(_) == list) else 0) for _ in tree]) + 1

#i = -10
#print(final_perspectives[0])#(perspectives[i])
#treeHeight(final_perspectives[0])#(perspectives[i])

In [70]:
for matchLabel, matchSets in matches.items():
    for matchSet in matchSets:
        print("{} :- {}".format(matchLabel, matchSet))
        subset = inp[matchSet[0]:matchSet[1]]
        print(inp[:matchSet[0]] + [[matchLabel, subset]] + inp[matchSet[1]:])

#subset = inp[matches[matchLabel][0][0]:matches[matchLabel][0][1]]
#print(inp[:matches[matchLabel][0][0]] + [[matchLabel, subset]] + inp[matches[matchLabel][0][1]:])


In [72]:
from ete2 import Tree
t = Tree( "((a,b),c);" )
t.render("mytree.png", w=183, units="mm")

ModuleNotFoundError: No module named 'ete2'